# Redes Neurais Recorrentes

Long-Short Term Memory (LSTM) são um tipo de rede neural, mais especificamente um tipo de rede neural recorrente.

Redes neurais recorrentes possuem uma característica distinta de redes neurais feedforward. Ao invés da informação seguir um fluxo contínuo sempre em um direção (usualmente para "frente"), as RNNs passam a informação também de volta ("trás"). Isso permite que essas simulem uma memória, sendo capazes de lidar melhor com problemas que variam com o tempo, como é o nosso caso.

In [1]:
import torch
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from context import fakenews
from fakenews import preprocess as pre
import gensim
import math

## Tensores

É a principal estrutura de dado do framework. Ela é essencialmente um array numpy de 3 dimensões. Entretanto, com tensores é possível realizar operações diretamente na GPU aumentando a eficiência dos algoritmos.

In [2]:
torch.rand(2, 3, 2)

tensor([[[0.2361, 0.5672],
         [0.9832, 0.1733],
         [0.4146, 0.3074]],

        [[0.0220, 0.3436],
         [0.0297, 0.7120],
         [0.0346, 0.6582]]])

Temos suporte direto a operações aritméticas assim como no numPy. Podemos realizar adições (`+`), subtrações (`-`), multiplicação por escalar (`*`), transposição (`T` ou `transp()`), produto de hadarmard (`*`), produto interno (`@`), entre outros.

In [3]:
A = torch.ones(2, 3, 2, dtype=torch.int) * 2
I = torch.eye(3, 2, dtype=torch.int)

In [4]:
A

tensor([[[2, 2],
         [2, 2],
         [2, 2]],

        [[2, 2],
         [2, 2],
         [2, 2]]], dtype=torch.int32)

In [5]:
I

tensor([[1, 0],
        [0, 1],
        [0, 0]], dtype=torch.int32)

In [6]:
A + torch.ones(2, 3, 2, dtype=torch.int)

tensor([[[3, 3],
         [3, 3],
         [3, 3]],

        [[3, 3],
         [3, 3],
         [3, 3]]], dtype=torch.int32)

In [7]:
A * I

tensor([[[2, 0],
         [0, 2],
         [0, 0]],

        [[2, 0],
         [0, 2],
         [0, 0]]], dtype=torch.int32)

In [8]:
# A * I == A @ I

In [9]:
A @ torch.eye(2, dtype=torch.int)

tensor([[[2, 2],
         [2, 2],
         [2, 2]],

        [[2, 2],
         [2, 2],
         [2, 2]]], dtype=torch.int32)

In [10]:
np.pi * A

tensor([[[6.2832, 6.2832],
         [6.2832, 6.2832],
         [6.2832, 6.2832]],

        [[6.2832, 6.2832],
         [6.2832, 6.2832],
         [6.2832, 6.2832]]])

In [11]:
rng = torch.rand(1, 3, 2)
A + rng

tensor([[[2.6728, 2.7987],
         [2.4019, 2.0513],
         [2.1012, 2.4703]],

        [[2.6728, 2.7987],
         [2.4019, 2.0513],
         [2.1012, 2.4703]]])

In [12]:
A.view(-1, 4)

tensor([[2, 2, 2, 2],
        [2, 2, 2, 2],
        [2, 2, 2, 2]], dtype=torch.int32)

A execução na GPU não ocorre de forma automática. Primeiro, podemos verificar se o dispositivo é suportado pelo PyTorch.

In [13]:
torch.cuda.is_available()

/home/thalesaguiar/.pyenv/versions/3.8.2/envs/fakenews.env/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


False

Em caso positivo, podemos especificar quais tensores terão suas operações executadas na placa gráfica, definir a execução de tudo por padrão.

In [14]:
# gpu = torch.device("cuda")
# A.to(gpu)
# cuda_tensor = torch.tensor([3, 4, 5], device=gpu)

In [15]:
a_view = A.view(-1, 3)
a_view

tensor([[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]], dtype=torch.int32)

In [16]:
A[0, 0, 0] = 99999
a_view

tensor([[99999,     2,     2],
        [    2,     2,     2],
        [    2,     2,     2],
        [    2,     2,     2]], dtype=torch.int32)

In [17]:
a_view[0, 2] = 55
A

tensor([[[99999,     2],
         [   55,     2],
         [    2,     2]],

        [[    2,     2],
         [    2,     2],
         [    2,     2]]], dtype=torch.int32)

## Rede Neural

O PyTorch permite criar modelos neurais com bastante facilidade e praticidade. Tanto a criação de modelos customizáveis e a grande quantidade de modelos prontos disponíveis é bastante simples. Vamos gerar um conjunto de dados aleatórios para experimentar as funções do framework.

In [18]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

X, Y = load_boston(return_X_y=True)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.3,
                                                shuffle=True)
Xdf = pd.DataFrame(X)
Xdf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


Transformando os dados para tensores, temos

In [19]:
Xtrain = torch.tensor(Xtrain, dtype=torch.float)
Xtest = torch.tensor(Xtest, dtype=torch.float)
Ytrain = torch.tensor(Ytrain, dtype=torch.float).view(-1, 1)
Ytest = torch.tensor(Ytest, dtype=torch.float).view(-1, 1)

Por exemplo, abaixo podemos criar uma rede neural simples apenas usando os tensores.

In [20]:
weights = torch.rand(1, 13, requires_grad=True)
weights

tensor([[0.8108, 0.2496, 0.2447, 0.8703, 0.5624, 0.8378, 0.0211, 0.3029, 0.2913,
         0.7112, 0.2858, 0.5990, 0.7198]], requires_grad=True)

In [21]:
bias = torch.rand(1, 1, requires_grad=True)
bias

tensor([[0.1822]], requires_grad=True)

O parâmetro `requires_grad` diz ao PyTorch que esse tensor deve ser levado em consideração quando algum otimizador for utilizado. Ou seja, no momento que fazemos a propagação do erro, esse tensor será atualizado.

Então, temos o seguinte problema:

$$y = D \times W + b$$

Onde $D$ é nossa base (apenas *features*), $W$ são os pesos, $b$ é um bias representando possíveis ruídos/erros obtidos das aproximações e $y$ são nossos valores esperados. Ou seja

$$\hat{y} = D \times W^{'} + b^{'}$$

Sendo $\hat{y}$ um valor próximo o suficiente de $y$, assim como $W^{'}$ e $b^{'}$.

In [22]:
yhat = Xtrain @ weights.T + bias
yhat[:3]

tensor([[737.9861],
        [533.0953],
        [769.7849]], grad_fn=<SliceBackward>)

In [23]:
def model(X):
    return X @ weights.T + bias

Agora, precisamos definir a função objetivo. Vamos usar um simples Mean Squared Error.

In [24]:
sqr_diff = (Ytrain - yhat) ** 2
torch.sum(sqr_diff) / sqr_diff.numel()

tensor(279559.8750, grad_fn=<DivBackward0>)

A operação `numel()` retorna a quantidade de elementos do tensor. Vamos criar uma função para calcular o erro

In [25]:
def mse(preds, real):
    diff = (real - preds) ** 2
    return torch.sum(diff) / diff.numel()

Para descobrir o quanto atualizar cada peso, vamos aplicar o *gradient descent*. 

In [26]:
loss = mse(Ytrain, yhat)
loss.backward()
weights

tensor([[0.8108, 0.2496, 0.2447, 0.8703, 0.5624, 0.8378, 0.0211, 0.3029, 0.2913,
         0.7112, 0.2858, 0.5990, 0.7198]], requires_grad=True)

Note que os valores do tensor não foram atualizados após executar o backpropagation. Isso é porque os valores do gradiente são armazenados no atributo `grad` de cada tensor

In [27]:
weights.grad

tensor([[5.0247e+03, 1.0381e+04, 1.2453e+04, 6.2112e+01, 5.8715e+02, 6.4833e+03,
         7.3192e+04, 3.6835e+03, 1.1492e+04, 4.5767e+05, 1.9161e+04, 3.6611e+05,
         1.3467e+04]])

Agora, podemos atualizar o tensor com o seu gradiente. Aqui devemos tomar cuidado, pois o PyTorch controla e memoriza todas as operações realizadas nos tensores com `require_grad=True` para utilizar no cálculo dos gradientes. Para atualizar os parâmetros, não queremos que essa operação seja gravada. Para evitar esse comortamento, podemos usar o operador de contexto `with` com a função `torch.no_grad()`.

In [28]:
# hyperparameters
weights.grad.zero_()
bias.grad.zero_()

nepochs = 250
lrate = 1e-15

for epoch in range(nepochs):
    predictions = model(Xtrain)
    loss = mse(predictions, Ytrain)
    loss.backward()
    if epoch % 10 == 0:
        print(f'{epoch:3}° {loss}')
    with torch.no_grad():
        weights -= weights.grad * lrate
        bias -= bias.grad * lrate
        weights.grad.zero_()
        bias.grad.zero_()

  0° 279559.875
 10° 279559.875
 20° 279559.875
 30° 279559.875
 40° 279559.875
 50° 279559.875
 60° 279559.875
 70° 279559.875
 80° 279559.875
 90° 279559.875
100° 279559.875
110° 279559.875
120° 279559.875
130° 279559.875
140° 279559.875
150° 279559.875
160° 279559.875
170° 279559.875
180° 279559.875
190° 279559.875
200° 279559.875
210° 279559.875
220° 279559.875
230° 279559.875
240° 279559.875


In [29]:
pred = model(Xtest)
loss = mse(pred, Ytest)
loss

tensor(280549.0625, grad_fn=<DivBackward0>)

In [30]:
(Ytest - pred)[:3]

tensor([[-515.0153],
        [-416.5723],
        [-437.9969]], grad_fn=<SliceBackward>)

Esse comportamento é devido a estratégia para automaticamente gerar o gradiente de qualquer estrutura. O PyTorch (e provavelmente outros) transforma o gradiente descendente/backpropagation em um grafo onde cada nó é uma operação. Assim, ele só precisa gerar a derivada de cada nó e aplicar o backpropagation.

### Otimizador

Também podemos selecionar qual tipo de otimizador usaremos para ajustar os parâmetros. O mais comum é o Stochastic Gradient Descent (SGD), mas o PyTorch oferece o Adadelta, Adagrad, RMSProp, entre vários outros dentro do pacoto `torch.optim`. Além disso, claro, também podemos customizar e criar o nosso próprio otimizador.

In [31]:
?torch.optim.RMSprop

Init signature:
torch.optim.RMSprop(
    params,
    lr=0.01,
    alpha=0.99,
    eps=1e-08,
    weight_decay=0,
    momentum=0,
    centered=False,
)
Docstring:     
Implements RMSprop algorithm.

Proposed by G. Hinton in his
`course <https://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf>`_.

The centered version first appears in `Generating Sequences
With Recurrent Neural Networks <https://arxiv.org/pdf/1308.0850v5.pdf>`_.

The implementation here takes the square root of the gradient average before
adding epsilon (note that TensorFlow interchanges these two operations). The effective
learning rate is thus :math:`\alpha/(\sqrt{v} + \epsilon)` where :math:`\alpha`
is the scheduled learning rate and :math:`v` is the weighted moving average
of the squared gradient.

Arguments:
    params (iterable): iterable of parameters to optimize or dicts defining
        parameter groups
    lr (float, optional): learning rate (default: 1e-2)
    momentum (float, optional): momen

Em geral, os otimizadores são variações do Gradient Descent/Backpropagation. Mas alguns são melhores para evitar cair em mínimos locais, ou mesmo para quantidade de épocas necessárias. Outros, como o Adam, podem aumentar a complexidade da otimização ao utilizar valores diferentes para cada parâmetro, consequentemente aumentando também a flexibilidade.

In [32]:
network = torch.nn.Linear(13, 1)
print(network.weight)
print(network.bias)

Parameter containing:
tensor([[ 0.2257,  0.0072, -0.1866,  0.0114,  0.2767,  0.1177, -0.1942,  0.2490,
          0.1854, -0.0676, -0.1980, -0.0742, -0.2446]], requires_grad=True)
Parameter containing:
tensor([0.2004], requires_grad=True)


In [33]:
norm = StandardScaler()
Xtrain_scl = torch.from_numpy(norm.fit_transform(Xtrain)).type(torch.float)
Xtest_scl = torch.from_numpy(norm.fit_transform(Xtest)).type(torch.float)

pred = network(Xtrain_scl.type(torch.float))
pred[:3]

tensor([[-0.5658],
        [ 0.4348],
        [-1.1127]], grad_fn=<SliceBackward>)

In [34]:
mse = torch.nn.functional.mse_loss
mse(pred, Ytrain)

tensor(608.3049, grad_fn=<MseLossBackward>)

In [35]:
optim = torch.optim.SGD(network.parameters(), lr=1e-5)
adam = torch.optim.Adam(network.parameters(), lr=1e-5)

## DataLoader

PyTorch permite encapsular a base de dados para melhor manipular o treinamento e batches da forma mais "pythonica" possível.

In [36]:
tensordata = torch.utils.data.TensorDataset(Xtrain_scl, Ytrain)
tensordata[:2]

(tensor([[ 1.1882, -0.4836,  1.0200, -0.2636,  0.5093, -1.4960,  1.0316, -0.8136,
           1.6651,  1.5283,  0.8325, -0.0765,  1.8228],
         [-0.2425, -0.4836,  1.2332, -0.2636,  0.4321,  1.6106,  0.8008, -0.8764,
          -0.5155, -0.0312, -1.6645,  0.1972, -1.5074]]),
 tensor([[12.],
         [50.]]))

In [37]:
batch_size = 25
dtl = torch.utils.data.DataLoader(tensordata, batch_size, shuffle=True)

## Treinamento

In [38]:
nepochs = 150
lrate = 1e-5

for epoch in range(nepochs):
    for Xbatch, Ybatch in dtl:
        pred = network(Xbatch)
        loss = mse(pred, Ybatch)
        adam.zero_grad()
        loss.backward(loss)
        adam.step()
    if epoch % 25 == 0:
        print(f'{epoch:3}° {loss}')

  0° 823.837890625
 25° 390.97283935546875
 50° 442.968994140625
 75° 729.6364135742188
100° 652.306640625
125° 308.9932556152344


# Redes custom

In [39]:
model = torch.nn.Sequential(
    torch.nn.Linear(13, 1),
    torch.nn.Flatten(0, 1)
)

In [40]:
class SigmoidNN(torch.nn.Module):
    
    def __init__(self, din, dh, dout):
        super().__init__()
        self.input = torch.nn.Linear(din, dh)
        self.hidden = torch.nn.Linear(dh, dout)
        self.out = torch.nn.Linear(dout, 1)
        self.activation = torch.nn.functional.sigmoid
        
    def forward(self, data):
        l1out = self.activation(self.input(data))
        l2out = self.activation(self.hidden(l1out))
        pred = self.out(l2out)
        return pred

In [41]:
x, y = load_boston(return_X_y=True)
x = torch.from_numpy(x).type(torch.float)
y = torch.from_numpy(y).type(torch.float)
x.shape

torch.Size([506, 13])

In [42]:
model = SigmoidNN(13, 50, 10)

criterion = torch.nn.functional.mse_loss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
for t in range(2000):
    y_pred = model(x)
    loss = criterion(y_pred.flatten(), y)
    if t % 100 == 99:
        print(t, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/home/thalesaguiar/.pyenv/versions/3.8.2/envs/fakenews.env/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


99 603.0927124023438
199 602.3561401367188
299 601.6265869140625
399 600.9038696289062
499 600.1842651367188
599 599.4631958007812
699 598.734375
799 597.9893798828125
899 597.2190551757812
999 596.4488525390625
1099 595.6712646484375
1199 594.8361206054688
1299 593.971923828125
1399 593.129150390625
1499 592.2615966796875
1599 591.36962890625
1699 590.432373046875
1799 589.4315795898438
1899 588.4310913085938
1999 587.3958129882812


## Rede Recorrente

Para redes recorrentes também temos uma classe pronta para ser usada. 

In [43]:
INPUT_SIZE = 13
HIDDEN_SIZE = 5
NLAYERS = 1
BATCH_SIZE = 1
SEQ_SIZE = 2

In [44]:
class RNNDual(torch.nn.Module):
    
    def __init__(self, din, dh, dbatch, dseq, nlayers):
        super().__init__()
        self.rnn = torch.nn.RNN(din, dh)
        self.lin = torch.nn.Linear(dh, 1)
        
    def forward(self, data):
        output, hidden = self.rnn(data)
        linout = self.lin(hidden[-1])
        return linout
        
rnnboston = RNNDual(INPUT_SIZE, HIDDEN_SIZE, BATCH_SIZE, SEQ_SIZE, 1)

O treinamento (backpropagation through time) é executado automaticamente com a seuinte instrução

In [45]:
Xtrain_scl.shape

torch.Size([354, 13])

In [46]:
Xtrain_seq = Xtrain_scl.view(BATCH_SIZE, -1, INPUT_SIZE)
Xtest_seq = Xtest_scl.view(BATCH_SIZE, -1, INPUT_SIZE)
print(f'Xtrain shape {Xtrain_seq.shape}')
print(f'Xtest shape {Xtest_seq.shape}')

Xtrain shape torch.Size([1, 354, 13])
Xtest shape torch.Size([1, 152, 13])


Com os dados ajustados para o formato do framework, vamos realizar o treinamento da rede.

In [47]:
def doc2tensor(document):
    for wrd in document:
        yield torch.from_numpy(wrd)

Usualmente, as classes do pacote `nn` são tratadas como layers. Ou seja, uma instância da RNN é um layer. Dessa forma, fica fácil gerar modelos complexos e altamente customizáveis. Por exemplo, podemos criar uma Rede Convolucional Recorrente entre outros franksteins.

In [48]:
rnnboston = torch.nn.RNN(INPUT_SIZE, HIDDEN_SIZE, BATCH_SIZE, SEQ_SIZE, 1)

In [49]:
def train(model, data):
    loss_fn = torch.nn.functional.mse_loss
    adam = torch.optim.Adam(model.parameters(), lr=5e-5)
    for epoch in range(30):
        out, hidden = model(data)
        break
        loss = loss_fn(out, Ytrain)    

In [50]:
train(rnnboston, Xtrain_seq)

TypeError: rnn_tanh() received an invalid combination of arguments - got (Tensor, Tensor, list, int, int, float, bool, bool, int), but expected one of:
 * (Tensor data, Tensor batch_sizes, Tensor hx, tuple of Tensors params, bool has_biases, int num_layers, float dropout, bool train, bool bidirectional)
      didn't match because some of the arguments have invalid types: ([32;1mTensor[0m, [32;1mTensor[0m, [31;1mlist[0m, [31;1mint[0m, [31;1mint[0m, [31;1mfloat[0m, [31;1mbool[0m, [32;1mbool[0m, [31;1mint[0m)
 * (Tensor input, Tensor hx, tuple of Tensors params, bool has_biases, int num_layers, float dropout, bool train, bool bidirectional, bool batch_first)
      didn't match because some of the arguments have invalid types: ([32;1mTensor[0m, [32;1mTensor[0m, [31;1mlist[0m, [31;1mint[0m, [32;1mint[0m, [32;1mfloat[0m, [32;1mbool[0m, [32;1mbool[0m, [31;1mint[0m)


In [ ]:
[torch.randn(1, 3) for _ in range(5)]

In [ ]:
# frankstein

### LSTM

Como vimos na parte teórica, a LSTM nada mais é que uma variação do "neurônio" de uma RNN.

![lstmhidden](imgs/lstmhidden.png)

O PyTorch também possui uma classe para o LSTM, tendo em vista sua popularidade.

In [ ]:
class LSTMFNews:
    
    def __init__(self, input_dim, hidden_dim, optim):
        self.embed = w2v
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim)
        self.output = torch.nn.Linear(hidden_dim, 1)
        
    def forward(data, targets, nepochs=100):
        tensordata = torch.TensorDataset(data, targets)
        dtl = DataLoader(tensordata)
        hidden = torch.rand(hidden_dim)
        
        vec = torch.tensor(self.embed.wv[data])
        out, hidden = self.lstm(vec.view(1, 1, -1), hidden)
        self.output()
                    
                

Na instância acima, definimos uma LSTM onde o *hidden state* possui dimensão $M\times N\times S$. Uma característica específica da LSTM nesse framework é a entrada esperada e o estado. Ambos devem ser tensores tridimensionais. Podemos treinar a rede ao propagar um elemento da sequência por vez

In [ ]:
# treina 1 em 1

Enquanto que para usar todos os elementos precisamos concatenar nossos dados.

In [ ]:
# todos de uma vez

Principalmente em NLP, a transformação do texto para representação numérica é bastante comum. Dessa forma, podemos criar uma LSTM customizada ao adicionar um layer de embedding que fica responsável por essa transformação.

In [ ]:
class LSTMfnews:
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Assim como podemos usar o novo layer durante o treinamento, aprendendo os embeddings de acordo com a task.

In [ ]:
# treina embedding + lstm

Claro que também podemos criar vários franksteins, da mesma forma que fizemos com a RNN. Note que o *gradient descent* é totalmente genérico. Isto é, ele não depende da arquitetura do seu modelo, apenas que as funções de ativação sejam deriváveis ou deriváveis por partes. Por isso podemos criar estruturas totalmente **idiotas**, com valores indo, voltando, pulando, ignorando, com muito layer, e com o que quisermos.

In [ ]:
# cria frankstein

É importante notar que redes profundas tendem a ocasionar *overfitting*, assim como os problemas decorrentes do algoritmo de aprendizado:

* Vanishing
* Exploding